# Design Doc

## Two main goals:
* shared memory parallelism
* readable code

It's hard to achieve these goals in Python -- you really want to have for loops, instead of allocating huge numpy arrays! In particular, in Python you're forced to rewrite variables in order to avoid allocations, which makes code really annoying to work with. The `mpi4py` use of xgpaint also makes it extremely wasteful for memory, forcing you to use nodes without using all of their cores.

### Some stats:
On a single Tiger node (40 Haswell cores per node), it takes ~1 minute to initialize the code, and then ~70 seconds per source realization for the full websky halo set. For comparison, Marcelo suggests that on cori (32 Haswell cores per node), it takes ~5 minutes on 16 nodes.

This corresponds to (5 minutes / 70 seconds) * (16 nodes / 1 node) * (32 cores / 40 cores) ~ **50x speedup**.

# 1. Read a Catalog

We operate with HDF5. I convert the `.pksc` format into `.hdf5` in the `pksc2hdf5.py` script (needs some work).

In [1]:
const NPROCS = 40

using Distributed
addprocs(NPROCS);

In [2]:
@everywhere begin
    using HDF5
    using SharedArrays
    using Healpix
    using PoissonRandom
    using Interpolations
    using QuadGK
    using Base.GC
    using Roots
    using Cosmology
    using Unitful
    using UnitfulAstro
    
    using Random
    # set different seeds for worker IDs
    Random.seed!(myid() + trunc(Int64, time()))
end

We load in the halos and then initialize some derived quantities.

In [3]:
@time begin
    hdata = h5open("/tigress/zequnl/xgpaint/websky_halos-light.hdf5", "r") do file
        read(file, "halos")
    end
    
    N_halos = size(hdata,2)
    
    halo_x_pos = SharedArray{Float32}(N_halos)
    halo_y_pos = SharedArray{Float32}(N_halos)
    halo_z_pos = SharedArray{Float32}(N_halos)
    halo_mass = SharedArray{Float32}(N_halos)
    
    # fill up known quantities
    halo_x_pos .= hdata[1,:]
    halo_y_pos .= hdata[2,:]
    halo_z_pos .= hdata[3,:]
    halo_mass .= hdata[4,:]
    hdata = 0
    @everywhere GC.gc()
    
    # derived parameters
    halo_comoving_dist = SharedArray{Float32}(N_halos)
    halo_redshift = SharedArray{Float32}(N_halos)
    
    halo_n_cen = SharedArray{Int32}(N_halos)
    halo_n_sat = SharedArray{Int32}(N_halos)
    halo_n_sat_bar = SharedArray{Float32}(N_halos)
end;

 51.767067 seconds (6.12 M allocations: 26.014 GiB, 0.91% gc time)


In the final version of this code, these parameters should be stored in a const julia structure passed around by reference.

In [4]:
@everywhere begin
    # output
    const nside   = 4096
    
    # jiang
    const jiang_gamma_1    = 0.13
    const jiang_alpha_1    = -0.83
    const jiang_gamma_2    = 1.33
    const jiang_alpha_2    = -0.02
    const jiang_beta_2     = 5.67
    const jiang_zeta       = 1.19

    # shang HOD
    const shang_Msmin  = 1e11
    const shang_Mmin   = 1e10
    const shang_Mpeak  = 10.0^12.2
    const shang_sigmaM = 0.4
    const shang_Td     = 24.4          #Planck 2013 values
    const shang_beta   = 1.75
    const shang_eta    = 3.2           #same as delta_CIB in paper
    const shang_I0     = 46
    
    # physical constants
    const phys_h     = 6.62606957e-27   # erg.s
    const phys_c     = 3e+10            # cm/s
    const phys_k     = 1.3806488e-16    # erg/K
    const phys_Msun  = 2e33             # g
    const phys_Mpc   = 3.086e24         # cm
    
    # cosmology
    const omegab  = 0.043
    const omegac  = 0.207
    const omegam  = omegab + omegac
    const h = 0.7
    const rhocrit = 2.78e11 * omegam * (h^2)
    
    const jl_cosmo = cosmology(h=h, OmegaM=omegam)
end

Core to the 

In [5]:
@everywhere function build_r2z_interpolator()
    """
    Construct a fast r2z linear interpolator.
    """
    zrange = LinRange(0.0, 6.0, 1000)
    rrange = [ustrip(comoving_radial_dist(u"Mpc", jl_cosmo, z))
        for z in zrange];
    r2z = LinearInterpolation(rrange, zrange);
    return r2z
end

@everywhere const r2z = build_r2z_interpolator()

In [6]:
function fill_halovars!(
        x, y, z, # inputs
        redshift_result, dist_result) # outputs
    """
    This function computes distance and redshift in parallel.
    """
    
    N_halos = size(x,1)
    @sync @distributed for i = 1:N_halos
        dist_result[i] = sqrt(x[i]^2 + y[i]^2 + z[i]^2)
        redshift_result[i] = r2z(dist_result[i])
    end
end

fill_halovars! (generic function with 1 method)

We compute the comoving distance, spherical coordinates, and redshift of the halos and store them in the shared memory array `halo_derived`. It's in parallel, so it takes just seconds!

In [7]:
@time fill_halovars!(halo_x_pos, halo_y_pos, halo_z_pos,
    halo_redshift, halo_comoving_dist)

  3.295952 seconds (1.99 M allocations: 101.625 MiB, 0.69% gc time)


Task (done) @0x00002b3f02d53340

In [8]:
@everywhere GC.gc()

# 2. HOD
We assign a number of satellites to each halo.

In [9]:
@everywhere function jiang_shmf(m,M_halo)
    dndm = (((jiang_gamma_1*((m/M_halo)^jiang_alpha_1))+
             (jiang_gamma_2*((m/M_halo)^jiang_alpha_2)))*
             (exp(-(jiang_beta_2)*((m/M_halo)^jiang_zeta))))
    return dndm
end

function build_shang_interp(min_log_M, max_log_M)
    
    x_m = LinRange(min_log_M, max_log_M, 1000)
    N_sat_i = zeros(size(x_m))

    function integrand_m(lm, lM_halo)
        # lm is natural log of m
        m = exp(lm)
        M_halo = exp(lM_halo)
        dns_dm = jiang_shmf(m,M_halo)
        return dns_dm
    end

    for i in 1:size(x_m,1)
        N_sat_i[i], err = quadgk( t->integrand_m(t, x_m[i]), 
                log(shang_Msmin), x_m[i], rtol=1e-8)
        N_sat_i[i] = max(0.0,N_sat_i[i])
    end

    return LinearInterpolation(x_m, N_sat_i)
end

function hod_shang(cen_result, sat_result, sat_bar_result,
        halo_mass::SharedArray)
    # computes shang HOD and generates a Poisson draw
    min_lm = log(minimum(halo_mass))
    max_lm = log(maximum(halo_mass))
    logM_to_N = build_shang_interp(min_lm, max_lm)
    N_halos = size(halo_mass,1)
    
    @sync @distributed for i = 1:N_halos
        cen_result[i] = 1
        
        sat_bar_result[i] = logM_to_N(log(halo_mass[i]))
        sat_result[i] = pois_rand(convert(Float64, sat_bar_result[i]))
    end
    
end

hod_shang (generic function with 1 method)

In [10]:
@time hod_shang(halo_n_cen, halo_n_sat, halo_n_sat_bar, halo_mass)

  9.618106 seconds (3.93 M allocations: 199.755 MiB, 2.17% gc time)


Task (done) @0x00002b3f03215ae0

# 3. Halos to Sources to Maps

This is a loop *for each halo*, which does the following.

```
for loop over $n_{halo}$
    for loop over $n_{sat}$ of the halo
        do tasks
```

### Tasks
1. draw a random satellite with position and mass
2. assign it a luminosity from that mass
3. convert that luminosity to a flux
4. map to a healpix pixel index and add that flux

In [11]:
@everywhere function mz2c(m,z)
    # concentration factor from Duffy et al. 2008
    return 7.85 * (m / (2e+12/h))^(-0.081) / (1+z)^0.71
end

@everywhere function m2r(m)
    return (3. * m / 4. / π / rhocrit)^(1. / 3.)
end

@everywhere begin
    f_nfw(x) = log(1.0+x) - x/(1.0+x)
    g(x,c,m) = m - f_nfw(x)/f_nfw(c)

    function solve_r(c,lnm)
        m = exp(lnm)
        return find_zero( x -> g(x,c,m) , (0.0,100.0), Bisection()) / c
    end

    function build_c_lnm2r_interp()
        """
        Generates an interpolator r(c, lnm)

        Generate a LinearInterpolation object that turns concentration
        and ln(M_halo) into satellite radius.
        """
        mmin = -7.1 # we have a minimum fractional mass of exp(-7)
        mmax = 0
        cmin = 1e-3
        cmax = 25.0

        cs  = LinRange(cmin,cmax,100)
        ms  = LinRange(mmin,mmax,100)
        r = [solve_r(c_,m_) for c_ in cs, m_ in ms]

        return LinearInterpolation((cs, ms), r)
    end
end

In [13]:
@everywhere random_phi() = rand() * 2π
@everywhere random_theta() = acos(2.0*rand()-1.0)

In [14]:
# surface brightness of centrals
@everywhere begin
    sigma_cen(m) = (exp( -(log10(m) - log10(shang_Mpeak))^2 / 
            (2*shang_sigmaM) ) * m) / sqrt(2π * shang_sigmaM)

    function nu2theta(nu)
        xnu = phys_h * nu / phys_k / shang_Td
        return xnu^(4.0 + shang_beta) / expm1(xnu) / nu / shang_I0
    end

    # <L_sat> interpolation values
    function integrand_L(lm,lM_halo)
        m      = exp(lm)
        return sigma_cen(m) * jiang_shmf(m,exp(lM_halo))
    end

    function build_sigma_sat_ln(min_ln_M, max_ln_M)
        """
        Pass the min and max of ln M, get a linear interpolator out
        that takes in ln(M_halo) and returns the surface brightness.
        """
        x = LinRange(min_ln_M, max_ln_M, 1000)
        L_mean = zeros(size(x))

        for i in 1:size(x,1)
            L_mean[i], err = quadgk( t->integrand_L(t, x[i]), 
                    log(shang_Mmin), x[i], rtol=1e-8)
        end
        return LinearInterpolation(x, L_mean, extrapolation_bc=0.0)
    end
end

In [15]:
max_m = maximum(halo_mass)
@everywhere const max_m_halo = $max_m

In [16]:
@everywhere const sigma_sat_ln = build_sigma_sat_ln( 
    log(shang_Mmin), log(max_m_halo))
@everywhere const c_lnm2r = build_c_lnm2r_interp()
@everywhere const Healpix_res = Resolution(nside)

In [17]:
@everywhere function l2f(Lum, r_comoving, redshift)
    return Lum / (4pi * r_comoving^2 * (1.0 + redshift))
end

In [18]:
@everywhere function integrand_muofn(lmu)
    mu=exp(lmu)
    dns_dm = jiang_shmf(mu,1.0)
    return dns_dm
end

@everywhere function build_muofn()
    mu = 10 .^ LinRange(-6,0,1000)
    n  = zeros(size(mu,1))
    for i in 1:size(mu,1)
        n[i], err = quadgk( integrand_muofn, 
                log(mu[i]), 0.0, rtol=1e-6)
    end

    return LinearInterpolation(reverse(n), reverse(mu))
end

In [19]:
@everywhere const muofn = build_muofn()

In [20]:
function make_map(x_cen, y_cen, z_cen, m_cen, d_cm_cen, redshift_cen, 
        n_sat, n_sat_bar, nu_obs)
    """Generate a CIB map given a HOD."""
    N_halos = size(m_cen,1)

    result_map = SharedArray{Float64}(Healpix.nside2npix(nside)); 
    result_map .= 0.0 # initialize to zero
    
    @sync @distributed  for i = 1:N_halos
        log_m_cen = log(m_cen[i])
        r_cen = m2r(m_cen[i])
        c_cen = mz2c(m_cen[i], redshift_cen[i])
        # concentration of satellites will be inherited from parent
        
        # compute central luminosity and flux
        nu  = (1+redshift_cen[i]) * nu_obs
        L_cen = sigma_cen(m_cen[i]) * nu2theta(nu) * (1+redshift_cen[i])^shang_eta
        f_cen = l2f(L_cen, d_cm_cen[i], redshift_cen[i])
        hp_index = Healpix.vec2pixRing(Healpix_res, x_cen[i], y_cen[i], z_cen[i])
        result_map[hp_index] += f_cen
        
        for j = 1:n_sat[i]
            # ================= Populate Centrals ==============
            log_msat_inner = max(log(rand()), -7.0) 
            r_sat = r_cen * c_lnm2r(c_cen, log_msat_inner) * 200^(-1. / 3.)
            m_sat = muofn(rand() * n_sat_bar[i]) * m_cen[i]
            # the 200 encodes that Rvir appearing in concentration is 
            # defined as 200 times the mean - Marcelo
            
            # now move the satellite exactly one R_sat away from the central
            phi = random_phi()
            theta = random_theta()
            x_sat = x_cen[i] + r_sat * sin(theta) * cos(phi)
            y_sat = y_cen[i] + r_sat * sin(theta) * sin(phi)
            z_sat = z_cen[i] + r_sat * cos(theta)
            d_sat = sqrt(x_sat^2 + y_sat^2 + z_sat^2)
            
            # ================= Halos to Sources  ==============
            redshift_sat = r2z(d_sat)
            nu  = (1+redshift_sat) * nu_obs
            L_sat = sigma_cen(m_sat) * nu2theta(nu) * (
                1+redshift_sat)^shang_eta
        
            
            f_sat = l2f(L_sat, d_sat, redshift_sat)
            
            hp_index = Healpix.vec2pixRing(Healpix_res, x_sat, y_sat, z_sat)
            result_map[hp_index] += f_sat
        end
    end
    
    pix_area = Healpix.nside2pixarea(nside)
    return result_map ./ pix_area
end

make_map (generic function with 1 method)

In [25]:
@time result = make_map(
    halo_x_pos, halo_y_pos, halo_z_pos, halo_mass, 
    halo_comoving_dist, halo_redshift,
    halo_n_sat, halo_n_sat_bar, 100e9
);

 71.557993 seconds (47.86 k allocations: 4.309 MiB)


In [22]:
# using Plots
# gr()  # Use the GR backend

# logm = Map{Float64, RingOrder}(nside)
# logm.pixels .= log10.(result);
# Plots.plot( logm )

In [23]:
run(`rm /tigress/zequnl/xgpaint/cib100.fits`)

Process(`rm /tigress/zequnl/xgpaint/cib100.fits`, ProcessExited(0))

In [24]:
m = Map{Float64, RingOrder}(nside)
m.pixels .= result;
Healpix.saveToFITS(m, "/tigress/zequnl/xgpaint/cib100.fits")